# IMDB mysql notebook

In [23]:
import mysql.connector
# widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
# files

 

import IMDBmedia as IM

IM.imdb['host'] = "localhost"
IM.imdb['user'] = "root"
IM.imdb['pass'] = "root"
IM.imdb['dbname'] = "IMDBmedia"
# Root directories for scanning media

# IMDB dirs
MovieCast = [
    '/export/Movies'
]

# the dir under this one will be based on media type, then ! title of movie/video/series
IM.Root = '/export/MediaCast'

IM.DF = True


sql = "SHOW TABLES"
IM.search_db(sql, ())



,Tables_in_IMDBmedia
0,movies_list
1,name.basics
2,series_list
3,title.akas
4,title.basics
5,title.crew
6,title.episode
7,title.principals
8,title.ratings


### This has many results

In [20]:

print ("Search for Movies and Videos:")
interact_manual(IM.search_title_basics, name='Star Wars%');


Search for Movies and Videos:


interactive(children=(Text(value='Star Wars%', description='name'), Button(description='Run Interact', style=B…

## Self Join with Sub Query example

In [4]:
IM.DF = True
print ("Search for TV Show Episode List (sub query, took around 32 min):")
List = IM.media_type('series')
interact_manual(IM.get_tv_name_year_type, show='Star Trek: Voyager', year=widgets.IntSlider(min=1950, max=2022, step=1, value=1995), MediaType=List);


Search for TV Show Episode List (sub query, took around 32 min):


interactive(children=(Text(value='Star Trek: Voyager', description='show'), IntSlider(value=1995, description=…

In [3]:
def two_step_tv_search(show, year, MediaType):
    IM.DF = False
    SDB = IM.get_imdb_dir_tv(show, year)
    for s in SDB:
        ID = s[0]
        break
    IM.DF = True
    print(ID)
    IM.get_tv_id(ID)
print ("Search for TV Show Episode List:")
List = IM.media_type('series')
interact_manual(two_step_tv_search, show='Star Trek: Voyager', year=widgets.IntSlider(min=1950, max=2022, step=1, value=1995), MediaType=List);

Search for TV Show Episode List:


interactive(children=(Text(value='Star Trek: Voyager', description='show'), IntSlider(value=1995, description=…

In [10]:
sql="""
SELECT B.tconst, N.nconst, concat(B.primaryTitle, ' (', B.startYear, ')') as title, N.primaryName, ML.file
FROM `title.basics` B, movies_list ML, 
    `title.principals` P, `name.basics` N
WHERE ML.titleId = B.tconst 
    AND ML.titleId = P.tconst 
    AND P.nconst = N.nconst
    AND P.category LIKE 'act%'
"""
val=()
IM.search_db(sql, val)


,tconst,nconst,title,primaryName,file
0,tt0015864,nm0000122,The Gold Rush (1925),Charles Chaplin,/export/MediaCast/movie/1925/The Gold Rush.mp4
1,tt0015864,nm0074788,The Gold Rush (1925),Henry Bergman,/export/MediaCast/movie/1925/The Gold Rush.mp4
2,tt0015864,nm0354913,The Gold Rush (1925),Georgia Hale,/export/MediaCast/movie/1925/The Gold Rush.mp4
3,tt0015864,nm0615306,The Gold Rush (1925),Tom Murray,/export/MediaCast/movie/1925/The Gold Rush.mp4
4,tt0015864,nm0841501,The Gold Rush (1925),Mack Swain,/export/MediaCast/movie/1925/The Gold Rush.mp4
...,...,...,...,...,...
3749,tt9243946,nm2804503,El Camino: A Breaking Bad Movie (2019),Matt Jones,/export/MediaCast/movie/2019/El Camino; A Brea...
3750,tt9777644,nm0005148,Da 5 Bloods (2020),Delroy Lindo,/export/MediaCast/movie/2020/Da 5 Bloods.mp4
3751,tt9777644,nm0507592,Da 5 Bloods (2020),Norm Lewis,/export/MediaCast/movie/2020/Da 5 Bloods.mp4
3752,tt9777644,nm0676370,Da 5 Bloods (2020),Clarke Peters,/export/MediaCast/movie/2020/Da 5 Bloods.mp4


In [22]:
sql="""
SELECT N.nconst, N.primaryName, count(N.nconst) as Movies, IMDBname(N.nconst) as link
FROM  movies_list ML, 
    `title.principals` P, `name.basics` N
WHERE ML.titleId = P.tconst 
    AND P.nconst = N.nconst
    AND P.category LIKE 'act%'
Group by N.nconst
HAVING count(N.nconst) > 9
Order by count(N.nconst) DESC
"""
val=()
print("Number of Movies by Actors/Actress in Movies List")
IM.search_db(sql, val)

Number of Movies by Actors/Actress in Movies List


,nconst,primaryName,Movies,link
0,nm0000329,Jackie Chan,56,https://www.imdb.com/name/nm0000329/
1,nm0000241,Jean-Claude Van Damme,40,https://www.imdb.com/name/nm0000241/
2,nm0000219,Steven Seagal,38,https://www.imdb.com/name/nm0000219/
3,nm0001472,Jet Li,34,https://www.imdb.com/name/nm0001472/
4,nm0000129,Tom Cruise,34,https://www.imdb.com/name/nm0000129/
5,nm0005458,Jason Statham,33,https://www.imdb.com/name/nm0005458/
6,nm0000093,Brad Pitt,31,https://www.imdb.com/name/nm0000093/
7,nm0000216,Arnold Schwarzenegger,27,https://www.imdb.com/name/nm0000216/
8,nm0000134,Robert De Niro,26,https://www.imdb.com/name/nm0000134/
9,nm0001191,Adam Sandler,26,https://www.imdb.com/name/nm0001191/


In [7]:
sql="""
SELECT B.tconst as ID, 
    E.seasonNumber as S, E.episodeNumber as Ep, 
    B.primaryTitle as title, B.startYear as year, B.originalTitle as org_title, 
    B.runtimeMinutes as minutes, B.titleType as video_type, IMDBurl(B.tconst) as imdb_link
        FROM `title.basics` B, `title.episode` E 
        WHERE B.tconst = E.tconst 
            AND E.parentTconst = 'tt0112178'
        Order by E.seasonNumber ASC, E.episodeNumber ASC;
"""
val=()
IM.DF=True
IM.search_db(sql, val)

,ID,S,Ep,title,year,org_title,minutes,video_type,imdb_link
0,tt0114530,1,1,Caretaker,1995,Caretaker,45,tvEpisode,https://www.imdb.com/title/tt0114530/
1,tt0708943,1,3,Parallax,1995,Parallax,46,tvEpisode,https://www.imdb.com/title/tt0708943/
2,tt0708996,1,4,Time And Again,1995,Time And Again,45,tvEpisode,https://www.imdb.com/title/tt0708996/
3,tt0708946,1,5,Phage,1995,Phage,46,tvEpisode,https://www.imdb.com/title/tt0708946/
4,tt0708978,1,6,The Cloud,1995,The Cloud,45,tvEpisode,https://www.imdb.com/title/tt0708978/
...,...,...,...,...,...,...,...,...,...
163,tt0708900,7,20,Friendship One,2001,Friendship One,43,tvEpisode,https://www.imdb.com/title/tt0708900/
164,tt0708934,7,21,Natural Law,2001,Natural Law,44,tvEpisode,https://www.imdb.com/title/tt0708934/
165,tt0708907,7,22,Homestead,2001,Homestead,44,tvEpisode,https://www.imdb.com/title/tt0708907/
166,tt0708957,7,23,Renaissance Man,2001,Renaissance Man,43,tvEpisode,https://www.imdb.com/title/tt0708957/
